# Indexing CSV dataset into Opensearch db
Before starting any query on OpenSearch Database, we need to "insert" data into the DB.
For OpenSearch DB, this is called "Indexing".

In this notebook, we will index a CSV dataset into DB and try out some fuzzy query.

> Note: we will use a public Japanese company name as the data source. You can download [here](https://info.gbiz.go.jp/hojin/DownloadTop).


----
## Startup OpenSearch

Run below command to start the docker-compose
```sh
docker-compose up
```

After the startup finished, open the URL "http://localhost:5601/" to access the dashboard.

Then, follow the [API](https://opensearch.org/docs/1.2/opensearch/rest-api/index-apis/create-index/) to create index.



-----
## Preparation

Let's start by installing necessary packages.

In [24]:
!pip install opensearch-py numpy pandas -q

In [25]:
from opensearchpy import OpenSearch, helpers
import numpy as np
import pandas as pd
import os
import unicodedata
import time
from pprint import pprint as pp

Let's define some paths:

In [48]:

# Point to the csv you download
csv_path = "./tmp/info_gbiz/Kihonjoho_UTF-8.csv"

# You can think like the Database name
index_name = "alias_concat_db_20230405"

Init a opensearch client.

In [27]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.
# ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Optional client certificates if you don't want to use HTTP basic authentication.
# client_cert_path = '/full/path/to/client.pem'
# client_key_path = '/full/path/to/client-key.pem'

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = False,
    # verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    # ca_certs = ca_certs_path
)


---- 
## Clean the input

The raw data sometimes included lots of special characters. We need to normalize and clean the unwanted words.


In [28]:
def remove_accented_chars(text):
#     ```
#     (NFKD) will apply the compatibility decomposition, i.e. 
#     replace all compatibility characters with their equivalents. 
#     ```
    text = unicodedata.normalize('NFKC', text)#.encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

# For example
remove_accented_chars('ＳＭｉＬＥ\u3000Ｎｉｓｅｋｏ\u3000Ｌａｎｇｕａｇｅ\u3000Ｓｃｈｏｏｌ合同会社')

'SMiLE Niseko Language School合同会社'

Load the csv data via Pandas

In [29]:
df = pd.read_csv(csv_path, dtype=str)
# we only need these fields:
df = df[["法人名", "郵便番号", "本社所在地"]]
df.sample(5)

,法人名,郵便番号,本社所在地
1940822,今村撚糸有限会社,9102334,福井県福井市大宮町第１４号４８番地
4338348,有限会社中部ゴム商会,4600022,愛知県名古屋市中区金山３丁目１３番１６号
2405558,有限会社芦田染工場,6040044,京都府京都市中京区小川通御池上る下古城町３８８番地
4643630,有限会社伸栄製茶,5121115,三重県四日市市堂ケ山町５０３番地の１
1136886,株式会社ルーク,1600023,東京都新宿区西新宿８丁目２番２１号


Some preprocessing to clean the data

In [30]:
# fill Na with empty string
df = df.fillna("")
df = df.applymap(remove_accented_chars)
# For 郵便番号, we want them in this format xxx-xxxx, if it is 7-digit
def add_hyphen(postal_code):
    if len(postal_code) == 7:
        return postal_code[:3] + '-' + postal_code[3:]
    else:
        return postal_code
df["郵便番号"] = df["郵便番号"].apply(add_hyphen)
df.sample(5)

,法人名,郵便番号,本社所在地
3229352,有限会社アド・スタッフ,822-0022,福岡県直方市知古2丁目6番6号
3371695,有限会社エヌ・エム・アシスト,862-0913,熊本県熊本市東区尾ノ上2丁目24番10号
1710181,DDS・ジャパン株式会社,227-0063,神奈川県横浜市青葉区榎が丘47番地34
676561,株式会社小野建設,353-0002,埼玉県志木市中宗岡3丁目3番5号
5123892,株式会社藤﨑プレス工業,890-0051,鹿児島県鹿児島市高麗町22番12号


Additionally, we want an extra column to store the concatenated name + postal code + address

In [31]:
df["concat_name"] = df["法人名"] + " " + df["郵便番号"] + " " + df["本社所在地"]
# Remove any leading/trailing space
df = df.applymap(str.strip)
df.sample(5)

,法人名,郵便番号,本社所在地,concat_name
3223445,有限会社クローバー福岡,811-2201,福岡県糟屋郡志免町桜丘4丁目21番7号,有限会社クローバー福岡 811-2201 福岡県糟屋郡志免町桜丘4丁目21番7号
2231362,合同会社尾張大文,491-0861,愛知県一宮市泉2丁目13番地9号,合同会社尾張大文 491-0861 愛知県一宮市泉2丁目13番地9号
2126015,特定非営利活動法人みんなのおしごと,432-8061,静岡県浜松市西区入野町4924番地の10佐鳴台パークホームズ205号,特定非営利活動法人みんなのおしごと 432-8061 静岡県浜松市西区入野町4924番地の1...
1786658,相鉄ホテル株式会社,220-0004,神奈川県横浜市西区北幸1丁目3番23号,相鉄ホテル株式会社 220-0004 神奈川県横浜市西区北幸1丁目3番23号
50168,株式会社ハウス工房コーワ,080-0021,北海道帯広市西十一条南17丁目4番地7,株式会社ハウス工房コーワ 080-0021 北海道帯広市西十一条南17丁目4番地7


Add the fields needed for indexing later. And rename column names for easier access

In [37]:
df["_index"] = index_name
# rename
df = df.rename(columns={
    '法人名': 'company_name',
    '郵便番号': 'postal_code',
    '本社所在地': 'address',
})
df.sample(5)

,company_name,postal_code,address,concat_name,_index
358876,TRUST株式会社,305-0821,茨城県つくば市春日3丁目13-6KASUGA32C105,TRUST株式会社 305-0821 茨城県つくば市春日3丁目13-6KASUGA32C105,alias_concat_db_20230405
5018978,株式会社山陰日本アルミ,683-0257,鳥取県米子市榎原146番地の61,株式会社山陰日本アルミ 683-0257 鳥取県米子市榎原146番地の61,alias_concat_db_20230405
3141569,伊予林商有限会社,793-0030,愛媛県西条市大町175番地2,伊予林商有限会社 793-0030 愛媛県西条市大町175番地2,alias_concat_db_20230405
72157,株式会社家庭サービス社,047-0037,北海道小樽市幸3丁目26番3号,株式会社家庭サービス社 047-0037 北海道小樽市幸3丁目26番3号,alias_concat_db_20230405
1486350,有限会社工房匠の館,164-0011,東京都中野区中央2丁目58番地10号ラックスタービル6階,有限会社工房匠の館 164-0011 東京都中野区中央2丁目58番地10号ラックスタービル6階,alias_concat_db_20230405


Convert the dataframe into chunks of list of dict.

In [38]:
# Split the DataFrame into chunks of given size
chunk_size = 50000
chunks = [chunk.to_dict(orient='records') for _, chunk in df.groupby(df.index // chunk_size)]
print(len(chunks))
print(len(chunks[0]))

104
50000


In [51]:
chunks[0][:2]

[{'company_name': '釧路検察審査会',
  'postal_code': '085-0824',
  'address': '北海道釧路市柏木町4-7',
  'concat_name': '釧路検察審査会 085-0824 北海道釧路市柏木町4-7',
  '_index': 'alias_concat_db_20230405'},
 {'company_name': '伊達簡易裁判所',
  'postal_code': '052-0021',
  'address': '北海道伊達市末永町47-10',
  'concat_name': '伊達簡易裁判所 052-0021 北海道伊達市末永町47-10',
  '_index': 'alias_concat_db_20230405'}]

Save memory...

In [23]:
del(df)

-----

## Indexing
Now the dataset is ready to be indexed.
Let's index them into the opensearch db by bulk importing.

In [39]:
print("Started import -----")
tic = time.perf_counter()
im_cn = 0

# Iterate over each chunk
for each_batch in chunks:
    # bulk import helper
    response = helpers.bulk(client, each_batch, max_retries=3)
    im_cn += 1
    toc = time.perf_counter()
    print(f"{im_cn} batches imported. Total elapsed time: {toc - tic:0.4f}sec.", end="\r")
    # reset the list

# total time
toc = time.perf_counter()
print(f"\nTotal time: {toc - tic:0.4f}sec, average {(toc - tic)/im_cn:0.4f}sec / batch of {chunk_size} rows.")
print("\nAll finished.")

Started import -----
104 batches imported. Total elapsed time: 890.7234sec.
Total time: 890.7236sec, average 8.5646sec / batch of 50000 rows.

All finished.


------

## Test query

In [58]:
df.sample(3)

,company_name,postal_code,address,concat_name,_index
1242115,有限会社エム・コーポレーション,124-0023,東京都葛飾区東新小岩2丁目7番6号恭永マンション105号室,有限会社エム・コーポレーション 124-0023 東京都葛飾区東新小岩2丁目7番6号恭永マン...,alias_concat_db_20230405
550531,神明神社,337-0015,埼玉県さいたま市見沼区大字蓮沼298番地,神明神社 337-0015 埼玉県さいたま市見沼区大字蓮沼298番地,alias_concat_db_20230405
3096460,学校法人村崎学園,770-0832,徳島県徳島市寺島本町東1丁目8番地,学校法人村崎学園 770-0832 徳島県徳島市寺島本町東1丁目8番地,alias_concat_db_20230405


Test connection by a fuzzy Search for the document.

In [62]:


query = {
  "query": {
    "match": {
      # "company_name": "簡易裁判所"
      # "concat_name": "簡易裁判所"
      "concat_name": "会社エム コーポレション 124-0023 東京都葛飾区東新小岩2丁目7番6号恭永マンション1O5号室"
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results:')


pp(response)


Search results:
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'RfQyUIcBLrfrvxLThmru',
                    '_index': 'full_gov_db_1',
                    '_score': 70.80829,
                    '_source': {'address': '東京都葛飾区東新小岩2丁目7番6号恭永マンション105号室',
                                'company_name': '有限会社エム・コーポレーション',
                                'concat_name': '有限会社エム・コーポレーション 124-0023 '
                                               '東京都葛飾区東新小岩2丁目7番6号恭永マンション105号室',
                                'postal_code': '124-0023'}},
                   {'_id': 'gO8xUIcBLrfrvxLThyy3',
                    '_index': 'full_gov_db_1',
                    '_score': 51.61158,
                    '_source': {'address': '東京都葛飾区東新小岩6丁目30番6号',
                                'company_name': '恭立スクリーン有限会社',
                                'concat_name': '恭立スクリーン有限会社 124-0023 '
                                               '東京都葛飾区東新小岩6丁目30番6号',
          

A more complex search:
- Exact match the postal code
- The result's postal_code should not be empty
- Then do a fuzzy search on concat name

And limit the output size to a given amount

In [ ]:

output_size = 3
postal_code = "108-0073"
concat_name = "株式会社コドモン 108-0073 東京都港区三田3-13"

query = {
    "size": output_size,
    "query": {
        "bool": {
        "must": [
            {"match": {"concat_name": concat_name}},
            {"match": {
                "postal_code": {
                    "query": postal_code,
                    "operator": "and"
                }
            }}
        ],
        "must_not": [
            {
                "match": {
                    "postal_code": {
                        "query": "",
                        "operator": "and"
                    }
                }
            }
        ]
        }
    }
}


response = client.search(
    body = query,
    index = index_name
)
print('\nSearch results:')


pp(response)